## Dependencies

In [1]:
!pip install clean-text
!pip install inflect


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
datatype = 'train' 

path = (
    'PIZZA_train.json' if datatype == 'train' else
    'PIZZA_dev.json' if datatype == 'dev' else
    'PIZZA_test.json' if datatype == 'test' else
    'unknown.json'  # Optional fallback
)

lines = (
    2457303 if datatype == 'train' else
    847 if datatype == 'dev' else
    1357 if datatype == 'test' else
    0  
)

print(f"Path: {path}")
print(f"Lines: {lines}")


Path: PIZZA_train.json
Lines: 2457303


## Parsing JSON File

In [4]:

#! load data given file path and number of line to load
def load_data(file_path,num_lines):
    data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline())) 
    return data
def load_random_data(file_path, num_lines, last_lines):
    data = []
    
    with open(file_path, 'r') as file:
        file.seek(0, 2)  # Move to the end of the file to get its size
        file_size = file.tell()
        
        # Read the last `last_lines` lines
        file.seek(max(file_size - 1024 * last_lines, 0))  # Optional: Adjust the seek window size for efficiency
        lines = file.readlines()
        last_lines_data = [json.loads(line.strip()) for line in lines[-last_lines:] if line.strip()]
        data.extend(last_lines_data)

        # Read `num_lines` random lines from the rest of the file (excluding the last `last_lines` lines)
        remaining_lines = file_size - sum(len(line) for line in lines[-last_lines:])
        for _ in range(num_lines):
            while True:
                # Pick a random position outside the last `last_lines` range
                random_pos = random.randint(0, remaining_lines - 1)
                file.seek(random_pos)
                
                # Read to the end of the current line to avoid partial lines
                file.readline()
                line = file.readline()  # Read the next line (complete line)
                if line:  # Ensure it's not an empty line
                    # Check if it's not from the last `last_lines` lines
                    # if any(line.strip() == last_line for last_line in last_lines_data):
                    #     continue  # Skip if it's from the last lines
                    try:
                        data.append(json.loads(line.strip()))
                        break  # Exit the loop for this line
                    except json.JSONDecodeError:
                        continue  # Skip if it's not valid JSON
    return data


In [ ]:
data=load_data(path,lines) if datatype=='train' else load_data(path,lines)
print(data[:-5])

In [6]:
#! split json data
def getextensions(datatype):
    if datatype=='train':
        return 'train.SRC','train.EXR','train.TOP'
    elif datatype=='dev':
        return 'dev.SRC','dev.EXR','dev.TOP'
    else:
        return 'test.SRC','test.EXR','test.TOP'
def get_training_data(data,datatype='train'):
    values=getextensions(datatype)
    training_data = []
    training_exr=[]
    training_top=[]
    # training_top_dec=[]
    for item in data:
        training_data.append(item[values[0]])  
        training_exr.append(item[values[1]])  
        training_top.append(item[values[2]])
        # if datatype=='train':
        #     training_top_dec.append(item[values[3]])  
    return training_data,training_exr,training_top
training_data,training_exr,training_top=get_training_data(data,datatype)   
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])


["i'd like a pizza with jalapeno peppers chicken and black olives without thin crust", "i'd like a pizza with sausages anchovies and jalapeno pepper without thin crust"]
['(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING JALAPENO_PEPPERS ) (TOPPING CHICKEN ) (TOPPING OLIVES ) (NOT (STYLE THIN_CRUST ) ) ) )', '(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING SAUSAGE ) (TOPPING ANCHOVIES ) (TOPPING JALAPENO_PEPPERS ) (NOT (STYLE THIN_CRUST ) ) ) )']
["(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza with (TOPPING jalapeno peppers ) (TOPPING chicken ) and (TOPPING black olives ) without (NOT (STYLE thin crust ) ) ) )", "(ORDER i'd like (PIZZAORDER (NUMBER a ) pizza with (TOPPING sausages ) (TOPPING anchovies ) and (TOPPING jalapeno pepper ) without (NOT (STYLE thin crust ) ) ) )"]


In [7]:
# Create an inflect engine
p = inflect.engine()

def replace_numbers_with_words(text):
    # Regular expression to find numbers in the text
    number_pattern = r'\b\d+\b'
    
    # Function to convert a number match to its word representation
    def number_to_words(match):
        number = int(match.group())
        # Convert to words and replace hyphens with underscores
        return p.number_to_words(number, andword="").replace('-', '_')
    
    # Substitute numbers with their word representation
    text = re.sub(number_pattern, number_to_words, text)
    
    # Replace the standalone 'a' with 'one'
    text = re.sub(r'\b[aA]\b', 'one', text)
    
    return text

In [8]:
#! a function to handle negations
def handle_negations(text):
    #! negations are based on training set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [9]:
#!# lemmatize words with all possible pos tags
def lem_word(word):
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [10]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords.update(not_stopwords)
# stopwords.discard('a')
# stopwords.discard('an')
# stopwords.discard('not')
# stopwords.discard('no')
# stopwords.discard('can')
# stopwords.discard('not_a')
# stopwords.discard('not_an')
# stopwords.discard('not_can')
# stopwords.discard('not_no')
stopwords=set()
# stopwords.add('and')
# stopwords.add('also')

In [ ]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    # training_data = [handle_negations(order) for order in training_data]
    training_data=[replace_numbers_with_words(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    # print(training_data)
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    #! remove "can" at the beginning of the sentence
    # training_data=[re.sub(r'^can\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # training_data = [[lemmatizer.lemmatize(word) for word in order] for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
print(training_data[:5])

[['i', 'like', 'one', 'pizza', 'with', 'jalapeno', 'pepper', 'chicken', 'and', 'black', 'olive', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'sausage', 'anchovy', 'and', 'jalapeno', 'pepper', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'caramelize', 'red', 'onion', 'pea', 'and', 'bean', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'peperoni', 'buffalo', 'sauce', 'and', 'pecorino', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'mozzarella', 'pepperoni', 'and', 'pesto', 'sauce', 'without', 'thin', 'crust']]


In [12]:
#! save processed training dataset
path = (
    'training_data_processed_1.txt' if datatype == 'train' else
    'dev_data_processed_1.txt' if datatype == 'dev' else
    'dev_data_processed.txt' if datatype == 'test' else
    'unknown_data_processed.txt'  # Optional fallback
)

with open(path, 'a') as f: #### dev_data_processed.txt
    for item in training_data:
        f.write("%s\n" % item)

In [13]:
#! a utility function for extra parentheses ) removal 
#! handles COMPLEX_TOPPING, NOT,... parenthesis cases
def remove_unmatched_parentheses(input_string):
    result = list(input_string)  # Convert to list for mutability
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
        elif char == ')' and i+2 >= len(result):
            result[i] = ''
            result[last_bracket_index] = ')'
    return ''.join(result)

In [14]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    # print(training_data)
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order=replace_numbers_with_words(order)
            order = re.sub(r"[\(\)]", "", order)
            order=word_tokenize(order) #! fix id and don't bugs
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            order=re.sub(r'\bd\s+', '', order) #! for d removal
            tokens = word_tokenize(order)
            # tokens = [lemmatizer.lemmatize(word) for word in tokens]
            tokens = [lem_word(word) for word in tokens]
            # print(tokens)
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)
print(order_category_labels[:5])
    
    
# def label_training_data(training_data, train_top):
#     labeled_data = []
    
#     for sentence_tokens, top in zip(training_data, train_top):
#         # Initialize labels for the sentence as "None"
#         labels = [None] * len(sentence_tokens)

#         # Regex to extract components from train.TOP
#         component_regex = r"\((NUMBER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY)\s+(.*?)\)"
#         components = re.findall(component_regex, top)

#         for component in components:
#             key = component[0].upper()
#             value = component[1].lower()

#             # Tokenize the value from train.TOP
#             value_tokens = word_tokenize(value)
#             print(value_tokens)
#             # Attempt to match the value tokens in the sentence tokens
#             for i in range(len(sentence_tokens)):
#                 # Match tokens sequentially
#                 if sentence_tokens[i:i + len(value_tokens)] == value_tokens:
#                     # Assign the label for each matching token
#                     for j in range(len(value_tokens)):
#                         if key == "SIZE":
#                             labels[i + j] = "SIZE"
#                         elif key == "STYLE":
#                             labels[i + j] = "STYLE"
#                         elif key == "TOPPING":
#                             labels[i + j] = "Topping"
#                         elif key == "NUMBER":
#                             labels[i + j] = "Number"
#                         elif key == "COMPLEX_TOPPING":
#                             labels[i + j] = "Complex-Topping"
#                         elif key == "QUANTITY":
#                             labels[i + j] = "Quantity"

#         # Append the labeled tokens for this sentence
#         labeled_data.append((sentence_tokens, labels))

#     return labeled_data

# # Get labeled data
# labeled_data_order_details = label_training_data(training_data, training_top)

# # Display the results
# for tokens, labels in labeled_data_order_details:
#     print("Tokens:", tokens)
#     print("Labels:", labels)
#     print()



 # loop over order category_labels and make each consecutive 0 -> first 0 -> SOP, last 0 -> EOP ,each consecutive 1 -> first 1 -> SOD, last 1 -> EOD
 # in between 0,1 or any 2 -> None
# def get_start_end_labels(order_category_labels):
#     order_labels = []
#     for item in order_category_labels:
#         order_labels.append([4] * len(item))
#         for i, label in enumerate(item):
#             if label == 0:
#                 if i == 0 or item[i - 1] != 0:
#                     order_labels[-1][i] = 0
#                 if i == len(item) - 1 or item[i + 1] != 0:
#                     order_labels[-1][i] = 1
#             elif label == 1:
#                 if i == 0 or item[i - 1] != 1:
#                     order_labels[-1][i] = 2
#                 if i == len(item) - 1 or item[i + 1] != 1:
#                     order_labels[-1][i] = 3
#     return order_labels

# order_category_labels=get_start_end_labels(order_category_labels)

 

[[2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [15]:
# import re
# from nltk.tokenize import word_tokenize

# def process_training_top(training_top, stopwords):
#     labeled_data = []

#     def remove_unmatched_parentheses(text):
#         stack = []
#         result = list(text)
#         for i, char in enumerate(text):
#             if char == '(':
#                 stack.append(i)
#             elif char == ')':
#                 if stack:
#                     stack.pop()
#                 else:
#                     result[i] = ''  # Remove unmatched closing parenthesis
#         for i in stack:  # Remove unmatched opening parentheses
#             result[i] = ''
#         return ''.join(result)

#     for i, item in enumerate(training_top):
#         sentence_tokens = []
#         labels = []
        
#         # Remove unwanted keywords like PIZZAORDER or DRINKORDER
#         unwanted_keywords = r'\(\b(ORDER|PIZZAORDER|DRINKORDER)\b'
#         cleaned_string = re.sub(unwanted_keywords, "", item)
#         # Remove stopwords and unnecessary parentheses
#         words = cleaned_string.split()
#         cleaned_words = [word for word in words if word.lower() not in stopwords]
#         cleaned_string = ' '.join(cleaned_words)
        
#         # Remove unmatched parentheses
#         cleaned_string = remove_unmatched_parentheses(cleaned_string)

#         # Regex to match nested parentheses
#         pattern_regex = r"\((\w+)\s+((?:[^\(\)]|\([^()]*\))*)\)"
#         matches = list(re.finditer(pattern_regex, cleaned_string))
        
#         in_not_block = False

#         for idx, match in enumerate(matches):
#             key, value = match.groups()
#             # Handle NOT block
#             if key == "NOT":
#                 in_not_block = True
#                 # Match nested content within the NOT block
#                 nested_match = re.match(pattern_regex, value.strip())
#                 if nested_match:
#                     nested_key, nested_value = nested_match.groups()
#                     print(f"Nested Match - Key: {nested_key}, Value: {nested_value}")
#                     value_tokens = word_tokenize(nested_value.strip())
#                     sentence_tokens.extend(value_tokens)
#                     labels.extend([f"NOT_{nested_key}"] * len(value_tokens))
#                 in_not_block = False  # Reset NOT block
#                 continue

#             value = re.sub(r'\(\s*\w+\s*', '', value)  # Match '(WORD)'
#             value = re.sub(r'[()]', '', value)  # Remove any remaining parentheses
#             # Tokenize the value
#             value_tokens = word_tokenize(value.strip())
#             sentence_tokens.extend(value_tokens)
#             # Labeling logic
#             if key == "STYLE":
#                 labels.extend(["STYLE"] * len(value_tokens))
#             elif key == "SIZE":
#                 labels.extend(["SIZE"] * len(value_tokens))
#             elif key == "TOPPING":
#                 labels.extend(["Topping"] * len(value_tokens))
#             elif key == "NUMBER":
#                 labels.extend(["Number"] * len(value_tokens))
#             elif key == "COMPLEX_TOPPING":
#                 labels.extend(["Complex-Topping"] * len(value_tokens))
#             elif key == "QUANTITY":
#                 labels.extend(["Complex-Topping"] * len(value_tokens))
#             else:
#                 labels.extend(["None"] * len(value_tokens))

#         # Add the processed tokens and labels for the current sentence
#         if sentence_tokens:
#             labeled_data.append((sentence_tokens, labels))

#     return labeled_data

# labeled_data = process_training_top(training_top, stopwords)

# # Output the labeled data
# for tokens, labels in labeled_data:
#     print("Tokens:", tokens)
#     print("Labels:", labels)


In [16]:
path = (
    'train_order_category_labels_1.txt' if datatype == 'train' else
    'dev_order_category_labels_1.txt' if datatype == 'dev' else
    'dev_order_category_labels.txt' if datatype == 'test' else
    'unknown_order_category_labels.txt'  # Optional fallback
)
for labels in order_category_labels:
    with open(path, 'a') as f: # dev
        f.write("%s\n" % labels)